In [2]:
import pandas as pd

df_articles = pd.read_csv("/Users/davislarson/Library/CloudStorage/OneDrive-BrighamYoungUniversity/BYU/WINTER 2025/455/data/shared_articles.csv")
df_interactions = pd.read_csv("/Users/davislarson/Library/CloudStorage/OneDrive-BrighamYoungUniversity/BYU/WINTER 2025/455/data/users_interactions.csv")

In [3]:
df_interactions.head()
df_articles.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en


In [4]:
df_interactions['rating'] = df_interactions['eventType'].map({
    'VIEW': 1,
    'LIKE': 2,
    'FOLLOW': 3,
    'BOOKMARK': 4,
    'COMMENT CREATED': 5,
})

df_interactions.head()

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,rating
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN,1
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US,1
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN,1
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN,3
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN,1


In [5]:
df_triple = df_interactions[['personId', 'contentId', 'rating']].copy()
df_triple

,personId,contentId,rating
0,-8845298781299428018,-3499919498720038879,1
1,-1032019229384696495,8890720798209849691,1
2,-1130272294246983140,310515487419366995,1
3,344280948527967603,310515487419366995,3
4,-445337111692715325,-7820640624231356730,1
...,...,...,...
72307,-9016528795238256703,-6590819806697898649,2
72308,102305705598210278,-5813211845057621660,1
72309,-9196668942822132778,-1999468346928419252,1
72310,-9016528795238256703,-6590819806697898649,1


In [6]:
# Drop any users that rated an article more than once keep their first
df_triple = df_triple.drop_duplicates(subset=['personId', 'contentId'], keep='first')


In [7]:
import numpy as np
from scipy.sparse import csr_matrix

# count the number of users and arcticles
U = df_triple['personId'].nunique()
A = df_triple['contentId'].nunique()

# create a dict to map the user and article ids to indices
user_mapper = dict(zip(np.unique(df_triple['personId']), list(range(U))))
article_mapper = dict(zip(np.unique(df_triple['contentId']), list(range(A))))

# create a dict to map the consecutive indices back to the original ids to access the data easily
user_inv_mapper = dict(zip(list(range(U)), np.unique(df_triple['personId'])))
article_inv_mapper = dict(zip(list(range(A)), np.unique(df_triple['contentId'])))

# create the list of each user and article to create the matrix
user_index = [user_mapper[i] for i in df_triple['personId']]
article_index = [article_mapper[i] for i in df_triple['contentId']]

X = csr_matrix((df_triple['rating'], (article_index, user_index)), shape=(A, U))

print(X)

  (0, 226)	2
  (0, 333)	1
  (0, 831)	1
  (0, 1142)	2
  (0, 1867)	1
  (1, 2)	2
  (1, 130)	1
  (1, 210)	1
  (1, 555)	1
  (1, 941)	1
  (1, 946)	2
  (1, 1149)	1
  (1, 1235)	5
  (1, 1499)	1
  (1, 1532)	1
  (2, 40)	1
  (2, 159)	1
  (2, 197)	1
  (2, 334)	1
  (2, 630)	1
  (2, 651)	1
  (2, 820)	1
  (2, 906)	1
  (2, 935)	2
  (2, 1004)	1
  :	:
  (2985, 798)	1
  (2985, 908)	1
  (2985, 978)	1
  (2985, 1079)	1
  (2985, 1136)	1
  (2985, 1221)	1
  (2985, 1248)	1
  (2985, 1263)	1
  (2985, 1319)	1
  (2985, 1334)	1
  (2985, 1343)	1
  (2985, 1383)	2
  (2985, 1404)	1
  (2985, 1405)	1
  (2985, 1527)	1
  (2985, 1597)	1
  (2985, 1728)	1
  (2985, 1833)	1
  (2985, 1874)	1
  (2985, 1888)	1
  (2986, 170)	1
  (2986, 588)	1
  (2986, 1004)	1
  (2986, 1237)	1
  (2986, 1597)	1


In [8]:
# THIS IS THE CODE TO GET A RECOMMENDATION FOR A GIVEN ARTICLE IN THE CONSOLE
# 
# import functions as f

# article_id = -3499919498720038879
# similar_ids, rec_distances = f.recommend(article_id, X, article_mapper, article_inv_mapper, k=5, messages=False)

# article_title = df_articles.loc[df_articles['contentId'] == article_id, 'title'].values[0]
# similar_titles = df_articles.loc[df_articles['contentId'].isin(similar_ids), 'title'].values

# print(f'If you liked article "{article_title}", you may also like the following articles:\n')
# for title in similar_titles:
# 	print(f"\t{title}")

In [ ]:
# for getting the recommendation given a matrix
def recommend(itemId, X, item_mapper, item_inv_mapper, k, metric='cosine', messages=True):
   from sklearn.neighbors import NearestNeighbors
   import numpy as np
   
   rec_ids = []                # Make a list for the recommended item IDs we'll get later
   item = item_mapper[itemId]  # Get the index of the movie ID passed into the function
   item_vector = X[item]       # Get the vector of user ratings for the movie ID passed into the function
   
   # Fit the clustering algorithm based on the user-item matrix X
   knn = NearestNeighbors(n_neighbors=k+1, algorithm="brute", metric=metric).fit(X)
   
   # Call the trained knn cluster model to return the nearest neighbors of the item_vector passed in
   rec = knn.kneighbors(item_vector.reshape(1,-1), return_distance=True)
   rec_indeces = rec[1][0]     # Parse out the list of indeces of the recommended items
   rec_distances = rec[0][0]   # Parse out the recommendation strength calculated as the distance from the cluster center
   rec_distances = np.delete(rec_distances, 0) # Drop the first number in the list because it is the distance of itemId from itself
   
   # We need to replace the recommended item indeces with their original item IDs
   for i in range(1, knn.n_neighbors): # n_neighbors is the number of neighbors to return
      rec_ids.append(item_inv_mapper[rec_indeces[i]])
      
   # It may help to see what this is. The distance list is first and the recommended item indeces are second
   if messages:
      print(f'List of recommended item indeces:\n{rec_indeces}\n')
      print(f'List of recommended item IDs:\n{rec_ids}\n')
      print(f'List of recommended item similarity to selected item:\n{rec_distances}\n')
   
   # Return two lists: the original item IDs of the recommendations and their similarity scores
   return rec_ids, rec_distances

In [9]:
def recommendModel(X, k, metric='cosine'):
   from sklearn.neighbors import NearestNeighbors
   
   # Fit the clustering algorithm based on the user-item matrix X
   knn = NearestNeighbors(n_neighbors=k+1, algorithm="brute", metric=metric).fit(X)
   
   return knn

In [ ]:
import joblib

# Save the model to a file for later use
collaborative_filtering_model = recommendModel(X, k=5, metric='cosine')

joblib.dump(collaborative_filtering_model, 'collaborative_filtering_model.sav')


['collaborative_filtering_model.sav']

In [ ]:
model_loaded = joblib.load('model.sav')

model_loaded.([X.iloc[0]])

In [ ]:
# import functions as f
# k=5

# # Ensure df_articles is indexed by contentId for efficient lookups
# df_articles = df_articles.set_index('contentId')

# df_recommendations = pd.DataFrame(columns=['If you liked'], index=article_mapper)
# for i in range(1, k+1):
#     df_recommendations[f'Article {i}'] = None

# for row in df_recommendations.itertuples():
#     rec_ids, rec_distances = f.recommend(row[0], X, article_mapper, article_inv_mapper, k=k, messages=False)

#     df_recommendations.at[row[0], 'If you liked'] = df_articles.at[row[0], 'title']

#     for i, r in enumerate(rec_ids):
#         df_recommendations.at[row[0], f'Article {i+1}'] = df_articles.at[r, 'title']
#         df_recommendations.at[row[0], f'Link {i+1}'] = df_articles.at[r, 'url']

# df_recommendations.to_json('recommendations.json', orient='index')
# Save the recommendations to a JSON file
# This will create a file named 'recommendations.json' with the recommendations
# in a format that can be easily used in a web application or for further analysis.
# The 'orient='index'' option will create a JSON object where each key is the
# article ID and the value is another object with the recommended articles.
# The resulting JSON file will have the following structure:
# {
#     "article_id_1": {
#         "If you liked": "Title of article 1",
#         "Article 1": "Title of recommended article 1",
#         "Article 2": "Title of recommended article 2",
#         ...
#         "Link 1": "URL of recommended article 1",
#         "Link 2": "URL of recommended article 2",
#         ...
#     },
#     "article_id_2": {
#         "If you read": "Title of article 2",
#         "Article 1": "Title of recommended article 1",
#         "Article 2": "Title of recommended article 2",
#         ...
#         "Link 1": "URL of recommended article 1",
#         "Link 2": "URL of recommended article 2",
#         ...
#     },
#     ...
# }

